## AAE is composed of:

- Prior distribution (Target distribution)
- AutoEncoder
  - Encoder (Generator)
  - Decoder
- Discriminator


x: input

z: latent vector (created by encoder)

p(x): model distribution

p_d(x): data distribution

q(z|x): AAE

q(z): (aggregated) posterior distribution

p(z): (arbitrary) prior distribution

AAE guides q(z) to match p(z)

$q(z) = \int_x q(z|x) p_d(x) dx$


In [13]:
import os
import math
import itertools
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
# from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tqdm import tqdm


In [3]:
os.makedirs("./images", exist_ok=True)

## Hyperparameters

In [4]:
channels = 1
img_size = 32
n_epochs = 100
batch_size = 64
lr = 0.002
b1 = 0.5
b2 = 0.999
latent_dim = 10 # output dim for encoder, input dim for decoder and discriminator
img_shape = (channels, img_size, img_size)

## Reparameterization

backprop을 하기 위해선 z들이 같은 분포를 가져야 함


In [5]:
def reparameterization(mu, logvar):
    std = torch.exp(logvar/2)
    sampled_z = Variable(Tensor(np.random.normal(0, 1, (mu.size(0), latent_dim))))
    z = sampled_z * std + mu
    return z

## Encoder

In [20]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        
        self.model = nn.Sequential(nn.Linear(1024, 512), 
                                   nn.Dropout(p=0.2), 
                                   nn.ReLU(), 
                                   nn.Linear(512, 512), 
                                   nn.Dropout(p=0.2), 
                                   nn.ReLU())
        
        self.mu = nn.Linear(512, latent_dim)
        self.logvar = nn.Linear(512, latent_dim)
        
    def forward(self, img):
        
        img_flat = img.view(img.shape[0], -1)
        
        x = self.model(img_flat)
        
        mu = self.mu(x)
        logvar = self.logvar(x)
        
        z = reparameterization(mu, logvar)
        return z

## Decoder

In [21]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        
        self.model = nn.Sequential(nn.Linear(latent_dim, 512), 
                                   nn.Dropout(p=0.2), 
                                   nn.ReLU(), 
                                   nn.Linear(512, 512), 
                                   nn.BatchNorm1d(512),
                                   nn.Dropout(p=0.2), 
                                   nn.ReLU(),
                                   nn.Linear(512, 1024),
                                   nn.Tanh())
        
    def forward(self, z):
        q = self.model(z)
        q = q.view(q.shape[0], *img_shape)
        return q

## Discriminator

In [22]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(nn.Linear(latent_dim, 512), 
                                   nn.Dropout(p=0.2), 
                                   nn.ReLU(), 
                                   nn.Linear(512, 256), 
                                   nn.Dropout(p=0.2), 
                                   nn.ReLU(),
                                   nn.Linear(256, 1),
                                   nn.Sigmoid())
        
    def forward(self, z):
        x = self.model(z)
        return x

## Loss function & Model initialization

In [23]:
adversarial_loss = torch.nn.BCELoss()
pixelwise_loss = torch.nn.L1Loss()

encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()

In [24]:
if torch.cuda.is_available():
    encoder.cuda()
    decoder.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    pixelwise_loss.cuda()

## Dataset & Dataloader

In [25]:
os.makedirs('./mnist', exist_ok=True)

dataset = datasets.MNIST(root='./mnist',
                         train=True,
                         transform=transforms.Compose([transforms.Resize(img_size), 
                                                       transforms.ToTensor(), 
                                                       transforms.Normalize([0.5], [0.5])]),
                         download=True)

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

## Training

In [26]:
def sample_image(n_row, epoch):
    z = Variable(Tensor(np.random.normal(0, 1, (n_row**2, latent_dim))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, f'images/gen_imgs_epoch_{epoch}.png', nrow=n_row, normalize=True)

In [28]:
optimizer_G = torch.optim.Adam(itertools.chain(encoder.parameters(), 
                                               decoder.parameters()), 
                               lr=lr, 
                               betas=(b1, b2))

optimizer_D = torch.optim.Adam(discriminator.parameters(), 
                               lr=lr, 
                               betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

for epoch in range(n_epochs):
    
    for imgs, _ in tqdm(dataloader):
        
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.), requires_grad=False)
        
        real_imgs = Variable(imgs.type(Tensor))
        
        
        # Generator
        optimizer_G.zero_grad()
        
        encoded_imgs = encoder(real_imgs)
        decoded_imgs = decoder(encoded_imgs)
        
        # Loss measures generator's ability to fool the discriminator
        g_loss = 0.001 * adversarial_loss(discriminator(encoded_imgs), valid) \
               + 0.999 * pixelwise_loss(decoded_imgs, real_imgs)
        
        g_loss.backward()
        
        optimizer_G.step()
        
        
        # Discriminator
        optimizer_D.zero_grad()
        
        # Sample noise as discriminator ground truth
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
        
        # Measures discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(z), valid)
        fake_loss = adversarial_loss(discriminator(encoded_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)
        
        d_loss.backward()
        
        optimizer_D.step()
    
    # Save result per epoch
    sample_image(n_row=10, epoch=epoch)
    print(f'epoch: {epoch}/{n_epochs} D loss: {d_loss.item()} G loss: {g_loss.item()}')


100%|█████████████████████████████████████████████████████████████| 938/938 [00:36<00:00, 25.78it/s]


epoch: 0/100 D loss: 0.5266318321228027 G loss: 0.13006426393985748


100%|█████████████████████████████████████████████████████████████| 938/938 [00:36<00:00, 25.49it/s]


epoch: 1/100 D loss: 0.5397289395332336 G loss: 0.10078063607215881


100%|█████████████████████████████████████████████████████████████| 938/938 [00:35<00:00, 26.23it/s]


...

epoch: 97/100 D loss: 0.4939908981323242 G loss: 0.1071738451719284


100%|█████████████████████████████████████████████████████████████| 938/938 [00:39<00:00, 23.78it/s]


epoch: 98/100 D loss: 0.3294450640678406 G loss: 0.10539760440587997


100%|█████████████████████████████████████████████████████████████| 938/938 [00:39<00:00, 23.88it/s]


epoch: 99/100 D loss: 0.3767692446708679 G loss: 0.12156082689762115
